# [ Settings ]

- Visit google drive (https://drive.google.com)
- Create a folder (example: hyu)
- Upload this entire folder (i.e., HYU-ECL3003) to the folder
- Open `train_detector.ipynb` on google drive (this action will open google colab session)
- Switch runtime to `T4 GPU`

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/')

# Move to the new directory
import os
os.chdir("/content/drive/MyDrive/hyu/HYU-ECL3003")

ModuleNotFoundError: No module named 'google.colab'

In [10]:
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached ultralytics-8.3.132-py3-none-any.whl (1.0 MB)
  Using cached onnxslim-0.1.52-py3-none-any.whl (145 kB)
ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)
ERROR: No matching distribution found for onnxruntime-gpu


In [1]:
import os
os.makedirs("data1", exist_ok=True)

import zipfile
with zipfile.ZipFile('/home/ircv11/gptiny/detection2.zip') as target_file:
  target_file.extractall('data1/')

os.system('echo "val: train.txt" >> data1/data1.yaml')

0

# [ Extract images ]

In [11]:
import cv2
import os

capture = cv2.VideoCapture("output_1.avi")
assert capture.isOpened(), "Cannot open the video file."

num_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

# Prepare folders
img_filename_fmt = 'data/images/train/frame_{:06d}.png'
dirname = os.path.dirname(img_filename_fmt)
os.makedirs(dirname, exist_ok=True)

for ii in range(num_frames):
    _, frame = capture.read()
    cv2.imwrite(img_filename_fmt.format(ii), frame)

capture.release()

In [ ]:
import cv2
import os

# 영상 열기
capture = cv2.VideoCapture("video/car1.avi")
assert capture.isOpened(), "영상을 열 수 없습니다."

# 원본 영상의 총 프레임 수와 FPS 확인
num_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
original_fps = capture.get(cv2.CAP_PROP_FPS)

# 타겟 FPS 설정 (10fps로 추출)
target_fps = 4
frame_interval = int(original_fps // target_fps)  # 몇 프레임마다 저장할지 계산

# 저장할 디렉토리 생성
img_filename_fmt = 'car_/images/train/cframe_{:06d}.png'
dirname = os.path.dirname(img_filename_fmt)
os.makedirs(dirname, exist_ok=True)

frame_index = 0
saved_index = 0

while True:
    ret, frame = capture.read()
    if not ret:
        break

    # 지정한 간격에 해당하는 프레임만 저장
    if frame_index % frame_interval == 0:
        filename = img_filename_fmt.format(saved_index)
        cv2.imwrite(filename, frame)
        saved_index += 1

    frame_index += 1

capture.release()


# [ Create symlink ]

In [ ]:
import os
os.symlink('data', 'datasets')

# [ Train a YOLO model ]

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("vehicle.pt")

# #Train the model on the COCO8 dataset for 100 epochs
# train_results = model.train(
#     data="/home/ircv11/gptiny/data/data.yaml",  # Path to dataset configuration file
#     epochs=20,  # Number of training epochs
#     imgsz=640,  # Image size for training
#     device=0,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
#     fliplr = 0.0,
#     hsv_h=0.0,
#     optimizer='Adam',
#     lr0=1e-2,
# )

New https://pypi.org/project/ultralytics/8.3.140 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.133 🚀 Python-3.10.12 torch-2.5.0 CUDA:0 (Orin, 7620MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/ircv11/gptiny/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimi

# [ Make an inference ]

In [ ]:
# Evaluate the model's performance on the validation set
metrics = model.val()

# Perform object detection on an image
results = model('/home/ircv11/HYU-ECL3003/rover/data/images/train/frame_000100.png')  # Predict on an image
results[0].show()  # Display results


image 1/1 /home/ircv11/HYU-ECL3003/rover/data/images/train/frame_000100.png: 384x640 3 persons, 1 chair, 1 laptop, 50.7ms
Speed: 2.7ms preprocess, 50.7ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:

# # Export the model to ONNX format for deployment
path = model.export(format="onnx")  # Returns the path to the exported model